In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df1 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df2 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
# Q1
df1.shape[1]

19

In [4]:
#Q2
df1['duration'] = (df1['tpep_dropoff_datetime'] - df1['tpep_pickup_datetime']).dt.total_seconds()/60
round(df1['duration'].std(),2)

42.59

In [5]:
df1['duration'].describe().round(2)

count    3066766.00
mean          15.67
std           42.59
min          -29.20
25%            7.12
50%           11.52
75%           18.30
max        10029.18
Name: duration, dtype: float64

In [6]:
#Q3
print(f"Rows before: {df1.shape[0]}")
total_rows = df1.shape[0]
df1 = df1[((df1['duration']>=1) & (df1['duration']<=60))]
print(f"Rows After: {df1.shape[0]}")
print(f"Fraction Remaining: {round(100*df1.shape[0]/total_rows)}%")

Rows before: 3066766
Rows After: 3009173
Fraction Remaining: 98%


In [7]:
#Q4
categorical = ['PULocationID', 'DOLocationID']

df1[categorical] = df1[categorical].astype(str)
train_dicts = df1[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape[1]

515

In [8]:
#Q5
# Create target
y_train = df1['duration'].values

# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Score it on RMSE:
round(mean_squared_error(y_train, regr.predict(X_train), squared=False),2)

7.65

In [9]:
#Q6
# Preprocess df2 & make duration
df2['duration'] = (df2['tpep_dropoff_datetime'] - df2['tpep_pickup_datetime']).dt.total_seconds()/60
df2 = df2[((df2['duration']>=1) & (df2['duration']<=60))]

# Apply dictvectorizer to df2
df2[categorical] = df2[categorical].astype(str)
val_dicts = df2[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)
y_val = df2['duration'].values

# Score it on RMSE:
round(mean_squared_error(y_val, regr.predict(X_val), squared=False),2)

7.81